TH Nürnberg - Neural Network Translator - Christoph Brandl, Philipp Grandeit

# PoC: Creation of a Frontend for PyTorch Framework

This jupyter notebook provides an exemplary implementation of the extraction of information from a neural network model created with the PyTorch framework.

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import scale
import json

# Building the network

A feed-forward network with different layer types is created.

In [ ]:
input_size = 8
hidden_size = 8
output_size = 1

# Build a feed-forward network
model = nn.Sequential(nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1)),
                      nn.AvgPool2d((3, 2), stride=(2, 1)),
                      nn.Linear(input_size, hidden_size, bias=False),
                      nn.ReLU(),
                      nn.Linear(hidden_size, output_size),
                      nn.Softmax())

# Save the network model

The functionality of saving models is available within the PyTorch framework.

In [ ]:
torch.save(model, 'pytorch_2_layer_diabetes.pt')

# Load the network model

The functionality for loading notebooks with the PyTorch framework is also available.

In [ ]:
z = torch.load('pytorch_2_layer_diabetes.pt')

# Read network structure and information from the loaded model

Now the information saved in the model can be read. First we have to verify that the loaded model contains all layer information we specified earlier. This can be done by using the print function.

In [ ]:
print(z)

As seen in the following step, the information from each layer can be accessed and, therefore, it is possible to create a json object which complies to the json intermediate format used for the Neural Network Translator.
This shows that models created with the PyTorch framework can be translated to the proposed intermediate file format and, thereby, it is possible to create a frontend plugin for the Neural Network Translator for the PyTorch framework.

In [ ]:
counter=0
output = { "class_name":"Sequential", "config":{"name":"sequential_1", "layers":[]}}
for layer in model:
    out_layer = dict()
    out_layer["class_name"] = str(type(layer).__name__)
    if (type(layer)==torch.nn.modules.activation.ReLU or type(layer)==torch.nn.modules.activation.Sigmoid
        or type(layer)==torch.nn.modules.activation.Tanh or type(layer)==torch.nn.modules.activation.Softmax):
        output["config"]["layers"][counter-1]["activation"] = str(type(layer).__name__).lower()
        out_layer = None
        counter = counter -1
    elif (type(layer)==torch.nn.modules.linear.Linear):
        if (counter==0):
            out_layer["batch_input_shape"] = ['null', layer.in_features]
        out_layer["units"]= layer.out_features
        out_layer["kernel_values"] = layer.weight.detach().numpy().tolist()
        if (layer.bias is not None):
            out_layer["bias_values"] = layer.bias.detach().numpy().tolist()
        out_layer["activation"]="linear"
    elif (type(layer)==torch.nn.modules.conv.Conv2d or type(layer)==torch.nn.modules.conv.Conv1d):
        if (counter==0):
            out_layer["batch_input_shape"] = layer.in_channels
        out_layer["kernel_size"] = list(tuple(layer.kernel_size))
        out_layer["strides"] = list(tuple(layer.stride))
        out_layer["padding"] = "valid" if layer.padding==0 else "same"
        out_layer["dilation"] = list(tuple(layer.dilation))
    elif (type(layer)==torch.nn.modules.pooling.MaxPool2d or type(layer)==torch.nn.modules.pooling.MaxPool1d
        or type(layer)==torch.nn.modules.pooling.AvgPool2d or type(layer)==torch.nn.modules.pooling.AvgPool1d):
        if (counter==0):
            out_layer["batch_input_shape"] = layer.in_channels
        out_layer["kernel_size"] = list(tuple(layer.kernel_size))
        out_layer["strides"] = list(tuple(layer.stride))
        out_layer["padding"] = "valid" if layer.padding==0 else "same"
        if (type(layer)==torch.nn.modules.pooling.MaxPool2d or type(layer)==torch.nn.modules.pooling.MaxPool1d):
            out_layer["dilation"] = [layer.dilation]

    if (out_layer is not None):
        output["config"]["layers"].append(out_layer)
    counter=counter+1

print(json.dumps(output))

### END